In [ ]:
# %matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table

from itertools import product
from copy import deepcopy as copy
from tqdm import tqdm
from lvmdrp.core.fit_profile import Gaussians
import bottleneck as bn

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}


f = np.sqrt(2 * np.pi)
def gaussians(pars, x):
    y = pars[0][:, None] * np.exp(-0.5 * ((x[None, :] - pars[1][:, None]) / pars[2][:, None]) ** 2) / (pars[2][:, None] * f)
    return np.sum(y, axis=0)

In [ ]:
# # create region files for fiber locations at column 2000
# _create_peaks_regions(fibermap=Table(drp.fibermap.data), column=2000)

In [ ]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)

In [ ]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")

In [ ]:
masters_flat = md.get_metadata(kind="master", imagetyp="flat").query("mjd == @masters_mjd").sort_values("camera")
masters_flat

In [ ]:
camera = "z2"
lamp = MASTER_CON_LAMPS[camera[0]]
if lamp == "ldls":
    counts_threshold = 20
elif lamp == "quartz":
    counts_threshold = 1000

mflat_path = path.full("lvm_master", drpver=drpver, tileid=1111, mjd=masters_mjd, kind=f"mflat_{lamp}", camera=camera)

centroids, trace_cent, trace_flux, trace_fwhm = imageMethod.trace_fibers(
    in_image=mflat_path,
    out_trace_amp="test_amp.fits", out_trace_cent="test_cent.fits", out_trace_fwhm="test_fwhm.fits", out_trace_cent_guess="test_cent_guess.fits",
    correct_ref=False, median_box=(1,10), coadd=5,
    counts_threshold=counts_threshold, max_diff=1.5, method="hyperbolic",
    ncolumns=(4086, 18), nblocks=18,
    fit_poly=False, poly_deg=(4, 4, 3), display_plots=True)

In [ ]:
from lvmdrp.functions import rssMethod


SLITMAP = Table(drp.fibermap.data)

fibermap = SLITMAP[SLITMAP["spectrographid"] == int(camera[1])]
select = fibermap["telescope"] == "Spec"
con_lamp = MASTER_CON_LAMPS[camera[0]]

trace_fwhm._mask[trace_fwhm._data <= 0] = True
# trace_fwhm.smoothTracePoly(6)

ifibers = np.where(select)[0]
for idx in ifibers:
    plt.figure(figsize=(15, 5))
    plt.plot(trace_fwhm._data[idx], color="r", label=fibermap[idx]["orig_ifulabel"])
    plt.plot(trace_fwhm._data[idx+1], color="k", label=fibermap[idx+1]["orig_ifulabel"])
    plt.plot(trace_fwhm._data[idx-1], color="k", label=fibermap[idx-1]["orig_ifulabel"])
    plt.title(f"{camera}")
    plt.legend(loc=2)


In [ ]:
mtrace_paths = []
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]
    
    lamp = MASTER_CON_LAMPS[camera[0]]
    if lamp == "ldls":
        counts_threshold = 20
    elif lamp == "quartz":
        counts_threshold = 1000
    
    # define master continuum exposure path
    mflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"mflat_{lamp}", camera=camera)
    # define master trace path
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    mtrace_paths.append(mtrace_path)

    # if os.path.isfile(mtrace_path):
    #     print(f"skipping {mtrace_path}, file already exist")
    #     continue
    
    # trace peak of fibers
    imageMethod.trace_peaks(in_image=mflat_path, out_trace=mtrace_path, steps=30, coadd=5,
                            threshold=counts_threshold, max_diff=1.5, method="gauss", median_box=10,
                            correct_ref=True, median_cross=1, poly_disp=4, write_trace_data=False, display_plots=True)
    break

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mtrace_paths)
md.add_masters(new_masters)

In [ ]:
from lvmdrp.core.tracemask import TraceMask
from lvmdrp.functions import rssMethod


SLITMAP = Table(drp.fibermap.data)

for camera in np.unique(masters_flat.camera.values):
    fibermap = SLITMAP[SLITMAP["spectrographid"] == int(camera[1])]
    select = fibermap["telescope"] == "Spec"
    con_lamp = MASTER_CON_LAMPS[camera[0]]

    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    trace = TraceMask()
    trace.loadFitsData(mtrace_path)
    trace._data[select]

    ifibers = np.where(select)[0]
    for idx in ifibers:
        plt.figure(figsize=(15, 5))
        plt.plot(trace._data[idx], color="r", label=fibermap[idx]["orig_ifulabel"])
        plt.plot(trace._data[idx+1], color="k", label=fibermap[idx+1]["orig_ifulabel"])
        plt.plot(trace._data[idx-1], color="k", label=fibermap[idx-1]["orig_ifulabel"])
        plt.title(f"{camera}")
        plt.legend(loc=2)
    break

In [ ]:
mwidth_paths = []
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]

    lamp = MASTER_CON_LAMPS[camera[0]]
    if lamp == "ldls":
        counts_threshold = 20
    elif lamp == "quartz":
        counts_threshold = 1000
    
    # define master continuum exposure path
    mflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"mflat_{lamp}", camera=camera)
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    # define master width path
    mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{camera}.fits")
    mwidth_paths.append(mwidth_path)

    if os.path.isfile(mwidth_path):
        print(f"skipping {mwidth_path}, file already exist")
        continue

    gaussian_fits, trace_flux, trace_cent, trace_fwhm, trace_flux_fit, trace_cent_fit, trace_fwhm_fit = imageMethod.trace_fiber_fwhm(in_image=mflat_path,
                                                                                                                                     in_centroid_trace=mtrace_path,
                                                                                                                                     out_fwhm_trace=mwidth_path,
                                                                                                                                     median_box=10, median_cross=1,
                                                                                                                                     ref_column=2000, nslices=18, nblocks=18, guess_fwhm=3,
                                                                                                                                     flux_threshold=None,
                                                                                                                                     fit_poly=False,
                                                                                                                                     display_plots=True)

    # evaluate continuum exposure model
    img = imageMethod.loadImage(mflat_path)
    mod = copy(img)

    column = np.arange(img._dim[0])
    for icolumn in tqdm(range(img._dim[1]), desc="evaluating Gaussians", unit="column", ascii=True):
        pars = (trace_flux_fit._data[:, icolumn], trace_cent_fit._data[:, icolumn], trace_fwhm_fit._data[:, icolumn] / 2.354)

        mod._data[:, icolumn] = gaussians(pars=pars, x=column)

    # save model and model to data ratio
    mmod_path = os.path.join(masters_path, f"lvm-mmod-{camera}.fits")
    mrat_path = os.path.join(masters_path, f"lvm-mrat-{camera}.fits")
    mod.writeFitsData(mmod_path)
    (mod / img).writeFitsData(mrat_path)

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mwidth_paths)
md.add_masters(new_masters)